In [1]:
import csv
import sys
import os
import numpy as np
import torch
import torch.nn as nn
from torch.nn import functional as F
import math
import copy
import random

sys.path.append('./backbones/ms-tcn')

from batch_gen import BatchGenerator
from model import MultiStageModel, Trainer

from src.utils import load_meta, eval_txts
from src.predict import predict_backbone
import configs.mstcn_config as cfg

In [2]:
random.seed(0)
np.random.seed(0)
torch.manual_seed(0)
torch.cuda.manual_seed_all(0)
torch.backends.cudnn.deterministic=True

In [3]:
device = 'cuda'

In [4]:
dataset = 'breakfast'     # choose from gtea, 50salads, breakfast
split = 4                 # gtea : 1~4, 50salads : 1~5, breakfast : 1~4
model_name = 'mstcn' # always "mstcn" in this notebook

In [5]:
actions_dict, \
num_actions, \
gt_path, \
features_path, \
vid_list_file, \
vid_list_file_tst, \
sample_rate,\
model_dir,\
result_dir, \
record_dir = load_meta(cfg.dataset_root, cfg.model_root, cfg.result_root, cfg.record_root, dataset, split, model_name)

Created :./model/mstcn/breakfast/split_4
Created :./result/mstcn/breakfast/split_4
Created :./record/mstcn/breakfast


In [6]:
batch_gen = BatchGenerator(num_actions, actions_dict, gt_path, features_path, sample_rate)
batch_gen.read_data(vid_list_file)

In [7]:
trainer = Trainer(cfg.num_stages, cfg.num_layers, cfg.num_f_maps, cfg.features_dim, num_actions)

In [8]:
trainer.train(save_dir=model_dir, 
              batch_gen=batch_gen, 
              num_epochs=cfg.num_epochs, 
              batch_size=cfg.batch_size, 
              learning_rate=cfg.lr, 
              device=device)

[epoch 1]: epoch loss = 10.747054,   acc = 0.178759
[epoch 2]: epoch loss = 8.398700,   acc = 0.260935
[epoch 3]: epoch loss = 7.209361,   acc = 0.345142
[epoch 4]: epoch loss = 6.285273,   acc = 0.435515
[epoch 5]: epoch loss = 5.685692,   acc = 0.496011
[epoch 6]: epoch loss = 5.030966,   acc = 0.577811
[epoch 7]: epoch loss = 4.680658,   acc = 0.619756
[epoch 8]: epoch loss = 4.910701,   acc = 0.617824
[epoch 9]: epoch loss = 4.265646,   acc = 0.666753
[epoch 10]: epoch loss = 4.130522,   acc = 0.686332
[epoch 11]: epoch loss = 3.867655,   acc = 0.703172
[epoch 12]: epoch loss = 3.543190,   acc = 0.747318
[epoch 13]: epoch loss = 3.708411,   acc = 0.714595
[epoch 14]: epoch loss = 3.673288,   acc = 0.741693
[epoch 15]: epoch loss = 3.383565,   acc = 0.769568
[epoch 16]: epoch loss = 2.934927,   acc = 0.800202
[epoch 17]: epoch loss = 3.508108,   acc = 0.771311
[epoch 18]: epoch loss = 2.959808,   acc = 0.797502
[epoch 19]: epoch loss = 3.261470,   acc = 0.795484
[epoch 20]: epoch lo

In [8]:
max_epoch = -1
max_val = 0.0
max_results = dict()

f = open(os.path.join(record_dir, 'split_{}_all.csv'.format(split)), 'w')

writer = csv.writer(f, delimiter='\t')
writer.writerow(['epoch', 'accu', 'edit', 
                 'F1@{}'.format(cfg.iou_thresholds[0]),
                 'F1@{}'.format(cfg.iou_thresholds[1]), 
                 'F1@{}'.format(cfg.iou_thresholds[2])])

for epoch in range(1, cfg.num_epochs+1):
    print('======================EPOCH {}====================='.format(epoch))
    predict_backbone(model_name, trainer.model, model_dir, result_dir, features_path, vid_list_file_tst, 
                     epoch, actions_dict, device, sample_rate)    
    results = eval_txts(cfg.dataset_root, result_dir, dataset, split, model_name)
    
    writer.writerow([epoch, '%.4f'%(results['accu']), '%.4f'%(results['edit']),
                    '%.4f'%(results['F1@%0.2f'%(cfg.iou_thresholds[0])]),
                    '%.4f'%(results['F1@%0.2f'%(cfg.iou_thresholds[1])]),
                    '%.4f'%(results['F1@%0.2f'%(cfg.iou_thresholds[2])])])

    curr_val = sum([results[k] for k in results.keys()])
    max_val = max(max_val, curr_val)

    if curr_val == max_val:
        max_epoch = epoch
        max_results = results

print('EARNED MAXIMUM PERFORMANCE IN EPOCH {}'.format(max_epoch))
print(max_results)

f.close()

======================EPOCH 1=====================
Acc: 17.6441
Edit: 42.2859
F1@0.10: 37.5635
F1@0.25: 27.6093
F1@0.50: 14.8322
======================EPOCH 2=====================
Acc: 33.1205
Edit: 43.4177
F1@0.10: 37.7586
F1@0.25: 27.0061
F1@0.50: 14.3214
======================EPOCH 3=====================
Acc: 36.3646
Edit: 47.2096
F1@0.10: 43.7278
F1@0.25: 33.6738
F1@0.50: 19.8783
======================EPOCH 4=====================
Acc: 28.7218
Edit: 47.3107
F1@0.10: 43.6547
F1@0.25: 35.1299
F1@0.50: 23.8566
======================EPOCH 5=====================
Acc: 46.8350
Edit: 48.5497
F1@0.10: 46.0089
F1@0.25: 38.8606
F1@0.50: 26.0154
======================EPOCH 6=====================
Acc: 51.1534
Edit: 54.6897
F1@0.10: 48.3756
F1@0.25: 41.2466
F1@0.50: 28.8420
======================EPOCH 7=====================
Acc: 53.8951
Edit: 54.5734
F1@0.10: 48.5545
F1@0.25: 41.8378
F1@0.50: 29.1833
======================EPOCH 8=====================
Acc: 59.7032
Edit: 55.0112
F1@0.10: 53.4999
F1

In [1]:
f = open(os.path.join(record_dir, 'split_{}_best.csv'.format(split)), 'w')
writer = csv.writer(f, delimiter='\t')
writer.writerow(['epoch', 'accu', 'edit', 
                 'F1@{}'.format(cfg.iou_thresholds[0]),
                 'F1@{}'.format(cfg.iou_thresholds[1]), 
                 'F1@{}'.format(cfg.iou_thresholds[2])])
writer.writerow([max_epoch, '%.4f'%(max_results['accu']), '%.4f'%(max_results['edit']),
                '%.4f'%(max_results['F1@%0.2f'%(cfg.iou_thresholds[0])]),
                '%.4f'%(max_results['F1@%0.2f'%(cfg.iou_thresholds[1])]),
                '%.4f'%(max_results['F1@%0.2f'%(cfg.iou_thresholds[2])])])
f.close()

NameError: name 'os' is not defined